In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("../datasets/exercises/downloaded_covid_effects.csv")

In [3]:
df.head()

,Unnamed: 0,Direction,Year,Date,Weekday,Current_Match,Country,Commodity,Transport_Mode,Measure,Value,Cumulative
0,0,Exports,2015,31/01/2015,Saturday,01/02/2020,All,All,All,$,257000000,257000000
1,1,Exports,2015,01/02/2015,Sunday,02/02/2020,All,All,All,$,123000000,380000000
2,2,Exports,2015,02/02/2015,Monday,03/02/2020,All,All,All,$,176000000,556000000
3,3,Exports,2015,03/02/2015,Tuesday,04/02/2020,All,All,All,$,115000000,671000000
4,4,Exports,2015,04/02/2015,Wednesday,05/02/2020,All,All,All,$,74000000,746000000


In [4]:
#Cogemos exportaciones en dólares
df = df[(df['Measure']=='$') & (df['Direction']=='Exports')]

In [5]:
#dummy_weekday = pd.get_dummies(df['Weekday'], prefix= "Day")
dummy_country = pd.get_dummies(df['Country'], prefix = "Country")
#dummy_year = pd.get_dummies(df['Year'], prefix = "Year")
dummy_commodity = pd.get_dummies(df['Commodity'], prefix = "Com")
dummy_transport = pd.get_dummies(df['Transport_Mode'], prefix = "Transport")

In [6]:
#df_new = df.join(dummy_country).join(dummy_weekday).join(dummy_year).join(dummy_commodity)
df_new = df.join(dummy_country).join(dummy_commodity).join(dummy_transport)

In [7]:
df_new.head()

,Unnamed: 0,Direction,Year,Date,Weekday,Current_Match,Country,Commodity,Transport_Mode,Measure,...,Com_All,Com_Dairy,Com_Forestry,Com_Fruit,Com_Meat,Com_Non-food manufactured goods,Com_Seafood,Transport_Air,Transport_All,Transport_Sea
0,0,Exports,2015,31/01/2015,Saturday,01/02/2020,All,All,All,$,...,1,0,0,0,0,0,0,0,1,0
1,1,Exports,2015,01/02/2015,Sunday,02/02/2020,All,All,All,$,...,1,0,0,0,0,0,0,0,1,0
2,2,Exports,2015,02/02/2015,Monday,03/02/2020,All,All,All,$,...,1,0,0,0,0,0,0,0,1,0
3,3,Exports,2015,03/02/2015,Tuesday,04/02/2020,All,All,All,$,...,1,0,0,0,0,0,0,0,1,0
4,4,Exports,2015,04/02/2015,Wednesday,05/02/2020,All,All,All,$,...,1,0,0,0,0,0,0,0,1,0


In [8]:
#Se elimina una de las dummies (la primera en este caso) para evitar la colinealidad
#feature_cols = (dummy_country.columns.values.tolist()[1:] + dummy_weekday.columns.values.tolist()[1:] +
#                    dummy_year.columns.values.tolist()[1:] + dummy_commodity.columns.values.tolist()[1:])
feature_cols = (dummy_country.columns.values.tolist()[1:] 
                + dummy_commodity.columns.values.tolist()[1:]  
                + dummy_transport.columns.values.tolist()[1:] )

In [9]:
X = df_new[feature_cols]
Y = df_new['Value']

In [10]:
lm = LinearRegression()
lm.fit(X,Y)

LinearRegression()

In [11]:
print(lm.intercept_)
print(lm.coef_)

16100520.833338827
[-7.44469981e+07 -3.30109232e+07 -6.34048106e+07 -8.33230397e+07
 -8.90068939e+07  1.73019603e+07 -9.40277272e+07 -5.93134043e+07
 -5.73317347e+07 -7.19519531e+07 -8.90631439e+07 -5.77437500e+07
 -6.23964772e+07 -7.87221358e+07  8.21740814e+07  1.18989583e+08]


In [12]:
list(zip(feature_cols, lm.coef_))

[('Country_Australia', -74446998.05173798),
 ('Country_China', -33010923.186870158),
 ('Country_East Asia (excluding China)', -63404810.55177842),
 ('Country_European Union (28)', -83323039.71845621),
 ('Country_Japan', -89006893.885109),
 ('Country_Total (excluding China)', 17301960.281573385),
 ('Country_United Kingdom', -94027727.21842319),
 ('Country_United States', -59313404.30176656),
 ('Com_Dairy', -57331734.66475366),
 ('Com_Forestry', -71951953.1249913),
 ('Com_Fruit', -89063143.88509755),
 ('Com_Meat', -57743749.9999966),
 ('Com_Non-food manufactured goods', -62396477.21842722),
 ('Com_Seafood', -78722135.80240409),
 ('Transport_All', 82174081.3850923),
 ('Transport_Sea', 118989583.33333075)]

In [13]:
lm.score(X,Y)

0.6706483857562637

El modelo puede ser escrito como el Intercept + el valor de cada característica (feature) por su coeficiente

In [14]:
df_new["Prediction"] = lm.predict(pd.DataFrame(df_new[feature_cols]))
df_new.head()

,Unnamed: 0,Direction,Year,Date,Weekday,Current_Match,Country,Commodity,Transport_Mode,Measure,...,Com_Dairy,Com_Forestry,Com_Fruit,Com_Meat,Com_Non-food manufactured goods,Com_Seafood,Transport_Air,Transport_All,Transport_Sea,Prediction
0,0,Exports,2015,31/01/2015,Saturday,01/02/2020,All,All,All,$,...,0,0,0,0,0,0,0,1,0,9.827460e+07
1,1,Exports,2015,01/02/2015,Sunday,02/02/2020,All,All,All,$,...,0,0,0,0,0,0,0,1,0,9.827460e+07
2,2,Exports,2015,02/02/2015,Monday,03/02/2020,All,All,All,$,...,0,0,0,0,0,0,0,1,0,9.827460e+07
3,3,Exports,2015,03/02/2015,Tuesday,04/02/2020,All,All,All,$,...,0,0,0,0,0,0,0,1,0,9.827460e+07
4,4,Exports,2015,04/02/2015,Wednesday,05/02/2020,All,All,All,$,...,0,0,0,0,0,0,0,1,0,9.827460e+07


In [15]:
SSD = np.sum((df_new['Prediction'] - df_new['Value'])**2)
SSD

3.134312051440592e+19

In [16]:
RSE = np.sqrt(SSD/(len(df_new) - len(feature_cols)-1))
RSE

27255942.65046168

In [17]:
values_mean = np.mean(df_new['Value'])
values_mean

32087613.72251706

In [18]:
error = RSE/values_mean
error

0.8494225493413735

Error muy alto. No es un buen modelo.
Yo creo que es porque no se ajusta bien a un modelo lineal.